In [1]:
import pandas as pd
from matchms.importing import load_from_msp
from matchms.logging_functions import set_matchms_logger_level
import itertools
from utils import load_spectra_metadata, get_matching_rows

set_matchms_logger_level("ERROR")

In [8]:
matches = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5.tsv", sep="\t")

In [2]:
def compute_combinations():
    predicted_spectra, predicted_spectra_metadata, predicted_spectra_names = load_spectra_metadata("../data/filtered/simulated_matchms_filter_1%I_all_peaks.msp", 'query')
    experimental_spectra, experimental_spectra_metadata, experimental_spectra_names = load_spectra_metadata("../data/experimental/RECETOX_GC-EI_MS_20201028.msp", 'reference')
    all_combinations = set(itertools.product(predicted_spectra_names, experimental_spectra_names))
    return all_combinations

In [3]:
def add_zeros(matches: pd.DataFrame, keys: list[str]) -> pd.DataFrame:
    all_combinations = compute_combinations()
    matches.set_index(keys, inplace=True)
    existing_rows = set(matches.index.to_list())
    rows_to_add = list(all_combinations - existing_rows)
    all_zeros = pd.DataFrame(0, index=rows_to_add, columns=matches.columns)
    merged = pd.concat([matches, all_zeros]).reset_index(names=keys)
    return merged

In [12]:
merged = add_zeros(matches, ['query', 'reference'])

In [13]:
merged.to_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5_with_0s_test.tsv", sep="\t", index=False)


In [14]:
matches_R = pd.read_csv("../data/output_matching/metaboannotation/metaboannotation_NA_to_zeros_all_peaks_filterNA.tsv", sep="\t")
merged_r = add_zeros(matches_R, ["name", "target_name"])
merged_r.to_csv("../data/output_matching/metaboannotation/metaboannotation_all_peaks_with_0s.tsv", sep="\t", index=False)

In [18]:
only_matching = get_matching_rows(merged_r, "name", "target_name")
only_matching.to_csv("../data/output_matching/metaboannotation/metaboannotation_all_peaks_with_0s_only_matching.tsv", sep="\t", index=False)

In [6]:
matches_matchms_top5 = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5_with_0s.tsv", sep="\t")
only_matching_matchms_top5 = get_matching_rows(matches_matchms_top5, "query", "reference")
only_matching_matchms_top5.to_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5_with_0s_only_matching.tsv", sep="\t", index=False)


In [7]:
matches_matchms_all_peaks = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_all_peaks_with_0s.tsv", sep="\t")
only_matching_matchms_all_peaks = get_matching_rows(matches_matchms_all_peaks, "query", "reference")
only_matching_matchms_all_peaks.to_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_all_peaks_with_0s_only_matching.tsv", sep="\t", index=False)
